# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
dict_attributes = set(dir(dict())) - set(dir(object()))

## Remark - Initializing dicts

There are 3 ways of initializing `dict`

- `dict(**kwargs)`: standard key/values
- `dict(Mapping, **kwargs)`:  If a mapping object is given, then
    1. A list of keys `list[key]` will be generated via `list(iter(Mapping))`
    2. The values will be looked up via `Mapping.__getitem__(key)`
- `dict(Iterable, **kwargs)`: If the first item is an iterable, then:
    1. A `list[tuple[key, value]]` will be generated via `list(iter(Iterable))`


In [ ]:
from collections.abc import Mapping, MutableMapping, Iterable, Union
from typing import Optional


def is_dunder(s: str) -> bool:
    return s.startswith("__") and s.endswith("__")


class Config(Iterable):
    def __init__(
        self, __dict__: Optional[Union[Mapping, Iterable]] = None, /, **kwargs
    ):
        super().__init__()

        if __dict__ is not None:
            assert not kwargs, f"kwargs not allowed if Mappping given!"

        items = kwargs if __dict__ is None else __dict__

        for key in items:
            value = items[key]
            if isinstance(value, Config):
                setattr(self, key, value)
            elif is_dunder(key):
                raise ValueError(f"Cannot set dunder key {key=}")
            # Recurse on Mapping
            else:
                if isinstance(value, Mapping):
                    setattr(self, key, Config(value))
                else:
                    setattr(self, key, value)

    def __str__(self):
        return self.__class__.__name__

    def __format__(self):
        return self.__class__.__name__

    def __repr__(self, nest_level: int = 0):
        print(nest_level)
        pad = r"_" * 4
        start_string = f"{self.__class__.__name__}("
        end_string = f")"

        lines = [start_string]

        for key, value in self.__dict__.items():
            if isinstance(value, Config):
                s = pad + f"{key} = {value.__repr__(nest_level+1)}"
            else:
                s = pad + f"{key} = {value}"
            lines.append(s)
        lines.append(end_string)
        result = ("\n" + pad * nest_level).join(lines)
        # print(result)
        return result

    def __len__(self):
        return self.__dict__.__len__()

    def __getitem__(self, key, from_iter=False):
        print(f"__getitem__ called from {id(self)} with {key=} and {from_iter=}")
        value = self.__dict__[key]

        if from_iter and isinstance(value, Config):
            return dict(value)
        return value

    def __iter__(self):
        print(f"__iter__ called, {id(self)=}")
        print(f"{self.__dict__=}")
        for key, value in self.__dict__.items():
            # if isinstance(value, Config):
            yield key, self.__getitem__(key, from_iter=True)

In [ ]:
Config()

In [ ]:
simple = Config(a=1, b=2)

dict(simple, c=1)

In [ ]:
z = Config(a=2, b=2, c=Config(x=1, y=2, z=Config(w=1, o=2)))

In [ ]:
list(iter(z.c.z))

In [ ]:
list(iter(z))

In [ ]:
dict(z)

In [ ]:
z.__dict__

In [ ]:
list(iter(z))

In [ ]:
dict(z)

In [ ]:
d = dict([(1, 2), (3, 4), (5, 6, 7)])

In [ ]:
dict(iter(d))

In [ ]:
dict(z)

In [ ]:
z.__dict__

In [ ]:
from typing import NamedTuple

In [ ]:
class MyTup(NamedTuple):
    count: int
    index: float

In [ ]:
MyTup(count=2, index=3)

In [ ]:
MyTup(count=2, index=3).count

In [ ]:
import torch